In [1]:
import __init__

Navigated to package root: /home/cyprien/CrystaLLMv2_PKV
Added package root to Python path


#### Jarvis-DFT
- Dataset: Source [JDFT](https://jarvis-materials-design.github.io/dbdocs/thedownloads/)
- Columns:
  - Database (manual)
  - **Material ID** (from source) 
  - Reduced Formula (pmg structure.composition.reduced_formula)
  - CIF (pmg - Cifwriter with symprec 0.1)

### Preprocess
- Random split 90:10 like the benchmark (no eval set)
- Generate XRD condition vectors using
  - pmg - XRDCalculator(wavelength="CuKa")
  - top 20 most intense peaks selected ($2\theta$ and int)
  - Normalisations
    - $2\theta$ min-max for 0,90
    - intensities min-max for 0,100
- Cleaned for CIF augmentation
  - note: filtered to context length for training etc. But for benchmarking we compare to all the structures in the database even if 16 were unparseable and 833 were above context length.
- saved HuggingFace as c-bone/jarvis-XRD

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(1)

df = pd.read_parquet('HF-databases/jarvis-XRD-processing-2/jarvis-XRD-unproc.parquet')

# Random Split for benchmarking because no train/test split provided in DiffactGPT paper
# 90% train, 10% test is the split used in paper
df['Split'] = np.random.choice(['train', 'test'], size=len(df), p=[0.9, 0.1])
df.to_parquet('HF-databases/jarvis-XRD-processing-2/jarvis-XRD-unproc.parquet', index=False)

# Make the test set parquet for benchmarking
df_test = df.copy()
df_test = df_test[df_test['Split'] == 'test']
df_test.to_parquet('_artifacts/jarvis-XRD/jarvis-test_ref.parquet', index=False)

In [11]:
!python _utils/_preprocessing/_calculate_XRD.py \
    --input_parquet HF-databases/jarvis-XRD-processing-2/jarvis-XRD-unproc.parquet \
    --output_parquet HF-databases/jarvis-XRD-processing-2/jarvis-XRD-unproc.parquet \
    --num_workers 32

Loading database from HF-databases/jarvis-XRD-processing-2/jarvis-XRD-unproc.parquet
Loaded 75993 entries
Processing XRD patterns for the entries
Generating XRD patterns: 100%|███| 75993/75993 [03:34<00:00, 354.67structures/s]
Computing condition vectors from XRD patterns
Processing XRD patterns: 100%|███| 75993/75993 [00:01<00:00, 57628.38patterns/s]
Theta range: 0-90, Intensity range: 0-100
Saving results to HF-databases/jarvis-XRD-processing-2/jarvis-XRD-unproc.parquet
done


In [21]:
!python _utils/_preprocessing/_cleaning.py \
    --input_parquet HF-databases/jarvis-XRD-processing-2/jarvis-XRD-unproc.parquet \
    --output_parquet HF-databases/jarvis-XRD-processing-2/jarvis-XRD-clean.parquet \
    --property_columns "['condition_vector']" \
    --num_workers 32 \
    --filter_to 1024

Loading data from HF-databases/jarvis-XRD-processing-2/jarvis-XRD-unproc.parquet as Parquet with zstd compression...

Normalizing property columns

Lets augment the CIFs now (parallelizing sometimes takes a min before speeding up
Number of CIFs before preprocessing: 75993
Number of workers: 32
100%|███████████████████████████████████| 75993/75993 [00:28<00:00, 2710.29it/s]
Number of CIFs before filtering out bad ones:  75993
Number of CIFs after filtering: 75977

Filtering dataframe of len 75977 to context length 1024
Tokenizer validation passed: token vocabulary is consistent.
Filtered dataframe length: 75144

Saving updated dataframe with len 75144 to HF-databases/jarvis-XRD-processing-2/jarvis-XRD-clean.parquet...
Preprocessing completed successfully.


In [23]:
!python _utils/_preprocessing/_save_dataset_to_HF.py \
    --input_parquet HF-databases/jarvis-XRD-processing-2/jarvis-XRD-clean.parquet \
    --output_parquet HF-databases/jarvis-XRD-processing-2/jarvis-XRD.parquet \
    --save_hub

Loading Hugging Face API key from API_keys.jsonc
Loading data from HF-databases/jarvis-XRD-processing-2/jarvis-XRD-clean.parquet as Parquet with zstd compression
Splitting dataset according to the 'Split' column
Train columns: ['Database', 'Material ID', 'Reduced Formula', 'CIF', 'condition_vector']
Uploading the dataset shards: 100%|███████████████| 1/1 [00:01<00:00,  1.24s/it]
Dataset saved to Hugging Face Hub as c-bone/jarvis-XRD


### Training

In [ ]:
!torchrun --nproc_per_node=2 _train.py --config '_config_files/training/conditional/xrd_studies/jarvis-xrd-slider-opt.jsonc'

### Generating

In [ ]:
!python _utils/_generating/make_prompts.py \
    --HF_dataset 'c-bone/jarvis-XRD' \
    --split 'test' \
    --automatic \
    --output_parquet '_artifacts/jarvis-XRD/jarvis-test_prompts.parquet' \
    --level 'level_3' \
    --condition_columns 'condition_vector'


Generating test split: 100%|█████| 7486/7486 [00:00<00:00, 222304.85 examples/s]
Loaded 7486 records from c-bone/jarvis-XRD (test split)

Generated automatic prompts at level_3
Using condition columns: ['condition_vector']

First 3 rows:
  Database Material ID Reduced Formula                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
!python _utils/_generating/generate_CIFs.py --config '_config_files/generation/conditional/xrd_studies/jarvis-xrd_eval.jsonc'

In [ ]:
!python _utils/_generating/postprocess.py \
    --input_parquet '_artifacts/jarvis-XRD/jarvis-test_gen.parquet' \
    --output_parquet '_artifacts/jarvis-XRD/jarvis-test_post.parquet' \
    --num_workers 32 \
    --column_name 'Generated CIF'

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/jarvis-XRD/jarvis-test_post.parquet' \
    --num_gens 20 \
    --ref_parquet '_artifacts/jarvis-XRD/jarvis-test_ref.parquet' \
    --output_parquet '_artifacts/jarvis-XRD/jarvis-test_metrics.parquet' \
    --num_workers 24

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/jarvis-XRD/jarvis-test_post.parquet' \
    --num_gens 1 \
    --ref_parquet '_artifacts/jarvis-XRD/jarvis-test_ref.parquet' \
    --output_parquet '_artifacts/jarvis-XRD/jarvis-test-1_metrics.parquet' \
    --num_workers 24

Metrics

In [5]:
from _utils import get_metrics
import pandas as pd

df_test = pd.read_parquet('_artifacts/jarvis-XRD/jarvis-test_ref.parquet')
df_metrics = pd.read_parquet('_artifacts/jarvis-XRD/jarvis-test_metrics.parquet')
metrics = get_metrics(df_metrics, n_test=len(df_test), only_matched=True)

Number of matched structures: 6534 / 7576
Mean RMS-d: 0.0162
Percent Matched (%): 86.25% (6534/7576)
a MAE: 0.1704
b MAE: 0.1629
c MAE: 0.2586
Volume MAE: 10.8239
a R^2: 0.9665
b R^2: 0.9709
c R^2: 0.9873
Volume R^2: 0.9355


In [ ]:
from _utils import get_metrics

df_metrics = pd.read_parquet('__old_files/_evaluation_files/conditional_studies/XRD/jarvis/comp-jarvis-XRD-test-rerun_metrics.parquet')
metrics = get_metrics(df_metrics, n_test=len(df_test), only_matched=True)

Number of matched structures: 6598 / 7576
Mean RMS-d: 0.0163
Percent Matched (%): 87.09% (6598/7576)
a MAE: 0.1557
b MAE: 0.1546
c MAE: 0.2439
Volume MAE: 9.9547
a R^2: 0.9715
b R^2: 0.9729
c R^2: 0.9890
Volume R^2: 0.9621


In [6]:
from _utils import get_metrics
import pandas as pd

df_test = pd.read_parquet('_artifacts/jarvis-XRD/jarvis-test_ref.parquet')
df_metrics = pd.read_parquet('_artifacts/jarvis-XRD/jarvis-test-1_metrics.parquet')
metrics = get_metrics(df_metrics, n_test=len(df_test), only_matched=True)

Number of matched structures: 4859 / 7576
Mean RMS-d: 0.0171
Percent Matched (%): 64.14% (4859/7576)
a MAE: 0.1104
b MAE: 0.1091
c MAE: 0.2036
Volume MAE: 8.3575
a R^2: 0.9860
b R^2: 0.9860
c R^2: 0.9889
Volume R^2: 0.9936


In [38]:
df_metrics = pd.read_parquet('__old_files/_evaluation_files/conditional_studies/XRD/jarvis/comp-jarvis-XRD-test-rerun-1gen_metrics.parquet')
metrics = get_metrics(df_metrics, n_test=len(df_test), only_matched=True)

Number of matched structures: 5028 / 7576
Mean RMS-d: 0.0169
Percent Matched (%): 66.37% (5028/7576)
a MAE: 0.1216
b MAE: 0.1223
c MAE: 0.2054
Volume MAE: 9.6020
a R^2: 0.9798
b R^2: 0.9799
c R^2: 0.9892
Volume R^2: 0.9456
